In [1]:
def read_hailstone(s):
    (a, b) = s.strip().split(" @ ")
    (a1, a2, a3) = [int(c.strip()) for c in a.split(",")]
    (b1, b2, b3) = [int(c.strip()) for c in b.split(",")]
    return ((a1, a2, a3), (b1, b2, b3))

In [4]:
hailstones = []

with open("input") as f:
    while (line := f.readline()):
        if line:
            hailstones.append(read_hailstone(f.readline()))


In [7]:
hailstones.sort(key=lambda x: x[0][0])

In [8]:
hailstones

[((11365561855318, 336994841944562, 75606016316333), (145, -33, 217)),
 ((31949340455546, 244722071566245, 188330495430550), (103, 64, 94)),
 ((65854129080671, 192592225528071, 24860313437127), (90, 122, 273)),
 ((78615637058877, 268918041816052, 426813479316490), (83, 40, -163)),
 ((80102417227421, 39742409350404, 268107592744246), (61, 278, 12)),
 ((95178319642971, 283655188828330, 292785831242096), (55, 24, -15)),
 ((96210228223541, 299931010941618, 272073444795304), (30, 8, 9)),
 ((96849558549729, 43756579183290, 37999549578007), (64, 287, 263)),
 ((119848610733977, 283837629736400, 296666563372742), (15, 24, -17)),
 ((123213451504129, 256367954091865, 186986187953457), (128, 60, 115)),
 ((133620200194889, 223581618915390, 124226525230672), (108, 104, 199)),
 ((143447577842385, 335373264673576, 296756828410190), (8, -32, -20)),
 ((144016749529449, 294333654224460, 295362735035170), (-26, 14, -13)),
 ((145092640710969, 255238260648483, 238813004877808), (104, 62, 44)),
 ((1512533793

In [11]:
"""
Based on hailstones, we can get some bounds.

Looking at x-coordinate: There are these extreme hailstones moving apart:
((144016749529449, 294333654224460, 295362735035170), (-26, 14, -13)),
((409853164949808, 283630297416922, 242168581023471), (41, -5, 45)),

So either:
- x-coordinate of start is <= 144_016_749_529_449 AND the velocity is > 41 (to catch up with the extremely far right one)
- x-coordinate of start is >= 409_853_165_949_808 AND the velocity is < -26 (so we catch up with the extremely far left one)

We can deduce more. In case (1), we actually need the x-velocity to be more than everything after that point.

Broadly, the rules here are:
(1) If our starting position w.r.t. some axis is X, and some hailstone (Y, V) exists with Y > X, then our velocity must be > V. 
(2) If our starting position w.r.t. some axis is X, and some hailstone (Y, V) exists with Y < X, then our velocity must be < V.

This ... feels like it's going to give us some bounds that can't be satisfied in some cases! Let's see.
"""
0

0

In [99]:
def dot(x, y):
    return sum(a*b for (a, b) in zip(x, y))

vec = (-1, 1, 3)
        
Xs = [dot(h[0], vec) for h in hailstones]
Xs = [(X - 1, X, X + 1) for X in Xs]

def allowed_ranges_for_velocity(X):
    lower = -1000000000000000000000000
    upper = 10000000000000000000000000

    for (p, v) in hailstones:
        if dot(p, vec) > X:
            lower = max(lower, dot(v, vec) + 1)
        if dot(p, vec) < X:
            upper = min(upper, dot(v, vec) - 1)
        if dot(p, vec) == X:
            lower = max(lower, dot(v, vec))
            upper = min(upper, dot(v, vec))
    return (lower, upper)

good_ranges = []
allowed = []
Xs = sorted([y for x in Xs for y in x])
for i, X in enumerate(Xs):
    (lower, upper) = allowed_ranges_for_velocity(X)
    allowed.append((X, lower <= upper, (lower, upper)))
    if i == 0 or i == len(Xs) - 1:
        print(f"{X:_}", lower, upper, lower <= upper)
    elif allowed[-1][2] != allowed[-2][2] and (allowed[-1][1] == True or allowed[-2][1] == True):
        print(f"{X:_}", lower, upper, lower <= upper)

60_905_669_367_581 1522 10000000000000000000000000 True
60_905_669_367_582 1522 1012 False
544_173_744_505_335 474 498 True
551_615_179_419_896 474 450 False
552_447_329_038_244 402 449 True
567_505_794_874_525 402 278 False
1_470_742_842_706_646 -1000000000000000000000000 -2623 True


In [94]:
float(2**50)

1125899906842624.0

OK. Let's assume I've (somehow) guessed the right velocity to use. That means I'm solving the equations
R + t_i * S = p_i + t_i*v_i

for unknown R, t_i. This is a linear equation with N+3 unknowns. So Gaussian elimination isn't going to be much use here - too many solutions. I want to do something Diophantine flavoured.

But if I've guessed S, then I can do something modulo something?